# Dividing melodies by track

In [1]:
import glob, os
import numpy as np
from music21 import converter, corpus, instrument, midi, note, chord, pitch, stream, interval, duration


In [2]:
import Dathoven

Using TensorFlow backend.
In C:\Users\jvela\miniconda3\envs\dathoven2\lib\site-packages\matplotlib\mpl-data\stylelib\Solarize_Light2.mplstyle: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
In C:\Users\jvela\miniconda3\envs\dathoven2\lib\site-packages\matplotlib\mpl-data\stylelib\Solarize_Light2.mplstyle: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
In C:\Users\jvela\miniconda3\envs\dathoven2\lib\site-packages\matplotlib\mpl-data\stylelib\Solarize_Light2.mplstyle: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
In C:\Users\jvela\miniconda3\envs\dathoven2\lib\site-packages\matplotlib\mpl-data\stylelib\Solarize_Light2.mplstyle: Passing (type, 1) or '1type' as a synonym of type is deprecated; 

In [3]:
midi_test_path = './YouBelongWithMe.mid'

In [20]:
def get_streams_from_midi_without_drums(midi_path):
    mf = midi.MidiFile()
    mf.open(midi_path)
    mf.read()
    mf.close()
    
    streams = []
    for track in mf.tracks:
        if (len(track.events) >0):
            track.events = [ev for ev in track.events if ev.channel != 10]
            if (len(track.events) > 0):
                streams.append(midi.translate.midiTrackToStream(track))          
    return streams

In [21]:
def from_midi_to_array_of_tracks(midi_path):
    streams = get_streams_from_midi_without_drums(midi_path)
    return [Dathoven.stream_to_array_of_pitches_with_time(stream) for stream in streams if len(stream.flat.notes) > 0]

In [22]:
def stream_to_array_of_pitches_with_time (stream):
    result = []
    offsets = []
    for item in stream.flat.notes:
        element = []
        if isinstance(item, note.Note):
            element = item
        else: # it's a chord
            #pitch = [note.pitch.ps for note in element.notes][0]
            element = item.notes[0]
        
        resultElement = {
            'absolute_offset': element._getOffset(), 
            'pitch': element.pitch.ps, 
            'duration': element.duration.quarterLength,
            'element': element
        }
        print(element.duration.quarterLength)
        
        if not resultElement['absolute_offset'] in offsets:  # only keep one note per time position
            if not resultElement['duration'] == 0:
                result.append(resultElement)
                offsets.append(resultElement['absolute_offset'])
    result.sort(key=lambda x: x['absolute_offset'])
    return result


In [23]:
def from_midi_to_tracks_intervals(midi_path):
    tracks = from_midi_to_array_of_tracks(midi_path)
    tracks_intervals = [Dathoven.from_pitches_to_intervals_with_time(track) for track in tracks if len(Dathoven.from_pitches_to_intervals_with_time(track)) > 0]
    return tracks_intervals

In [24]:
def get_folder_songs_tracks_intervals(folder_path):
    songs = []
    for file in glob.glob(folder_path):
        print("Reading file " + file)
        song = from_midi_to_tracks_intervals(file)
        print(f"{len(song)} tracks read")
        if (len(song) > 0):
            songs.append(song)
    return songs

In [25]:
songs_with_tracks = get_folder_songs_tracks_intervals("data/examples/*.mid")

Reading file data/examples\3LAU feat. Bright Lights - How You Love Me  (midi by Carlo Prato) (www.cprato.com).mid
2 tracks read
Reading file data/examples\Above & Beyond - Sun & Moon  (midi by Carlo Prato) (www.cprato.com).mid
5 tracks read
Reading file data/examples\Above & Beyond - Tri-state  (midi by Carlo Prato) (www.cprato.com).mid
3 tracks read
Reading file data/examples\Above & Beyond - We're All We Need (Original Mix) (midi by Carlo Prato) (www.cprato.com).mid
2 tracks read
Reading file data/examples\Above & Beyond feat. Alex Vargas - Blue Sky Action  (midi by Carlo Prato) (www.cprato.com).mid
2 tracks read
Reading file data/examples\Adam F - Circles  (midi by Carlo Prato) (www.cprato.com).mid
2 tracks read
Reading file data/examples\Aether & Enzalla - Elysia's Heart  (midi by Carlo Prato) (www.cprato.com).mid
1 tracks read
Reading file data/examples\Afrojack & David Guetta - Another Life  (midi by Carlo Prato) (www.cprato.com).mid
2 tracks read
Reading file data/examples\Alan 

In [26]:
len(songs_with_tracks)

399

In [27]:
songs_with_tracks[:2]

[[[{'relative_offset': 0.25, 'interval': 0.0, 'duration': 1},
   {'relative_offset': 1.5, 'interval': 0.0, 'duration': 1},
   {'relative_offset': 1.5, 'interval': 0.0, 'duration': 1}],
  [{'relative_offset': 0.25, 'interval': 0.0, 'duration': 1},
   {'relative_offset': 0.08333333333333331, 'interval': 0.0, 'duration': 1},
   {'relative_offset': 0.16666666666666669, 'interval': 3.0, 'duration': 1},
   {'relative_offset': 0.16666666666666663, 'interval': 0.0, 'duration': 1},
   {'relative_offset': 0.08333333333333337, 'interval': -5.0, 'duration': 1},
   {'relative_offset': 0.25, 'interval': 0.0, 'duration': 1},
   {'relative_offset': 0.25, 'interval': -2.0, 'duration': 1},
   {'relative_offset': 0.08333333333333326, 'interval': 2.0, 'duration': 1},
   {'relative_offset': 0.16666666666666674, 'interval': 0.0, 'duration': 1},
   {'relative_offset': 0.16666666666666674, 'interval': 2.0, 'duration': 1},
   {'relative_offset': 0.08333333333333326, 'interval': 0.0, 'duration': 1},
   {'relati

In [28]:
# Save into a pickle file.
import pickle

pickle.dump( songs_with_tracks, open( "songs_with_tracks.p", "wb" ) )